<a href="https://colab.research.google.com/github/villasen/colab_notebooks/blob/master/Tensorflow_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating And Running Tensorflow_lite model


In [0]:
!git clone https://github.com/villasen/Sound-Datasets.git

Cloning into 'Sound-Datasets'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 30921 (delta 0), reused 2 (delta 0), pack-reused 30918
Receiving objects: 100% (30921/30921), 1.70 GiB | 28.20 MiB/s, done.
Resolving deltas: 100% (6140/6140), done.
Checking out files: 100% (33680/33680), done.


In [0]:
!git clone https://github.com/villasen/ML-KWS-for-MCU.git

fatal: destination path 'ML-KWS-for-MCU' already exists and is not an empty directory.


In [0]:
!rm -r ML-KWS-for-MCU

In [0]:
from google.colab import files

In [0]:
import os
os.chdir('/content/ML-KWS-for-MCU')


In [0]:
os.chdir('/content/')

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import hashlib
import math
import os.path
import random
import re
import sys
import tarfile

import numpy as np
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

from tensorflow.contrib.framework.python.ops import audio_ops as contrib_audio
from tensorflow.python.ops import io_ops
from tensorflow.python.platform import gfile
from tensorflow.python.util import compat

import argparse

import input_data
import models


from tensorflow.python.platform import gfile

In [0]:
 def load_wav_file(filename):
  with tf.Session(graph=tf.Graph()) as sess:
    wav_filename_placeholder = tf.placeholder(tf.string, [])
    wav_loader = io_ops.read_file(wav_filename_placeholder)
    wav_decoder = contrib_audio.decode_wav(wav_loader, desired_channels=1)
    return sess.run(
        wav_decoder,
        feed_dict={wav_filename_placeholder: filename}).audio.flatten()
  
load_wav_file(filename)

In [0]:
%env FILE_PATH=Sound-Datasets/combined_datasets/Helicopter/1-172649-A.wav

In [0]:
!python ML-KWS-for-MCU/wav_to_features.py --window_size_ms 40.0 --window_stride_ms 20.0 --feature_bin_count 10 \
--input_wav $FILE_PATH --output_c_file wav_to_feat.c


In [0]:
!python wav_to_features.py --window_size_ms 40.0 --window_stride_ms 20.0 --feature_bin_count 10 \
--input_wav $FILE_PATH --output_c_file wav_to_feat.c

In [0]:
label_count=12
sample_rate=16000
clip_duration_ms=1000
window_size_ms=40.0
window_stride_ms=20.0
feature_bin_count=10
quantize=True
preprocess='mfcc'
input_wav='0b7ee1a0_nohash_3.wav'
output_c_file='done.c'


# Start a new TensorFlow session.
sess = tf.InteractiveSession()


model_settings = models.prepare_model_settings(
      10, 16000, 1000, 40.0, 20.0, 10,'mfcc')

#model_settings = models.prepare_model_settings(label_count, sample_rate,
#                clip_duration_ms, window_size_ms, window_stride_ms,
#               feature_bin_count, preprocess)

#print(model_settings)

audio_processor = input_data.AudioProcessor(None, None, 0, 0, '', 0, 0,
                                              model_settings,None)

results = audio_processor.get_features_for_wav(input_wav, model_settings,
                                                 sess)
features = results[0]

features_min, features_max = input_data.get_features_range(model_settings)





In [0]:
quantized_features = []

for value in features.flatten():
  quantized_value = int(round((255 * (value - features_min)) / (features_max - features_min)))
  if quantized_value < 0:
    quantized_value = 0
  if quantized_value > 255:
    quantized_value = 255
  quantized_features.append(quantized_value)

Reshape_1 = np.reshape(quantized_features, (49,10))  


In [0]:
Reshape_1.shape

(49, 10)

In [0]:
print(Reshape_1)

In [0]:

graph_def_file = "/content/speech_dscnn_172.pb"
input_arrays = ["Reshape_1"]
output_arrays = ["labels_softmax"]
input_shapes = {"Reshape_1": [1,49,10,1]}

converter = tf.lite.TFLiteConverter.from_frozen_graph(
  graph_def_file, input_arrays, output_arrays, input_shapes)
tflite_model = converter.convert()
open("converted_model2.tflite", "wb").write(tflite_model)
print(input_arrays)

['Reshape_1']


In [0]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="converted_model2.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[9.9218082e-01 1.4634981e-03 7.0478168e-04 5.6071795e-04 1.0309902e-03
  4.5320328e-04 4.8955966e-04 3.7163886e-04 6.6869549e-04 6.1295443e-04
  8.4630074e-04 6.1686942e-04]]
